<a href="https://colab.research.google.com/github/Jahan08/Amber-tutorial/blob/main/Retrieve-Data-prostate-cancer-data_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 4.7 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0


In [3]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [94]:
target = new_client.target
target_query = target.search('LAPC4')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,LAPC4,20.0,False,CHEMBL613292,[],CELL-LINE,9606


In [95]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL613292'

In [87]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [96]:
df = pd.DataFrame.from_dict(res)
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,117200,[],CHEMBL667652,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,0.0356
1,Not calculable,118456,[],CHEMBL668502,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,None
2,None,123345,[],CHEMBL667652,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,19.855
3,Not calculable,136075,[],CHEMBL668502,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,None
4,None,138488,[],CHEMBL667652,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,0.00112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8207,None,24422709,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5049248,Antiproliferative activity against human DU145...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,200.0
8208,None,24422710,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5049248,Antiproliferative activity against human DU145...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,200.0
8209,None,24513169,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5056340,Antitumor activity against human DU-145 cells ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,3.0
8210,None,24513170,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5056340,Antitumor activity against human DU-145 cells ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,DU-145,9606,None,None,IC50,uM,UO_0000065,None,3.0


In [97]:

# considering nM unit for MIC
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
len(df)

6158

In [98]:
df = df[df.canonical_smiles.notna()]
df = df[df.standard_value.notna()]

In [ ]:
# considering nM unit for MIC
df = df[(df['standard_units'] == '%')]
df = df[(df['relation'] == '=')]
len(df)

In [99]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i < 1000:
        STATUS.append("Yes") #active
        
    elif 20000 <= i <= 100000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

<ipython-input-99-92f3a7d8753f>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
0,None,117200,[],CHEMBL667652,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,DU-145,9606,None,None,IC50,uM,UO_0000065,None,35.60,Yes
2,None,123345,[],CHEMBL667652,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,DU-145,9606,None,None,IC50,uM,UO_0000065,None,19855.00,IN
4,None,138488,[],CHEMBL667652,In vitro cytotoxic activity against prostate c...,F,None,None,BAO_0000190,BAO_0000219,...,DU-145,9606,None,None,IC50,uM,UO_0000065,None,1.12,Yes


In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 80:
        STATUS.append("Yes") #active
        
    elif i == 0:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

In [100]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

2840

In [102]:
df.to_csv('bioactivity_data_LAPC4_IC50_20-100.csv', index=False)
     